## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.io
import itertools as it
import scipy.special as psi
plt.style.use('classic')
import seaborn as sns
import pandas as pd
import time 

from scipy.io import loadmat
from scipy import stats
from numpy.random import seed
from numpy.random import rand
from scipy.integrate import quad
from scipy.io import savemat
from tempfile import TemporaryFile
from scipy.io import loadmat
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.decomposition import KernelPCA
from mpl_toolkits import mplot3d
from mPE_fn import mPE
from scipy.spatial import distance
from scipy.stats import entropy

## Load and Clean Data

In [2]:
# loading entire high-dimensional trajectory
path = '/rds/general/user/lr4617/home/4th_Year_Project/CAPTURE_rat_multidimensional/raw_data/normal/'
trajectories = os.listdir(path)

s = 1

lengths = []
nan_cols = [12,13,14]
for n, traj_n in enumerate(trajectories):
    if traj_n != "traj_2":
        print(traj_n)
        path_n = path + traj_n + '/trajectories/'
        trajs = os.listdir(path_n)
        trajectory = loadmat(path_n + trajs[0])
        trajectory = trajectory['trajectory']
        # removing invalid values (e.g. NaN)
        # input data is already normalized (z-score) but needs to get rid of non-valued datapoints
        # all_trajectories = np.zeros((trajectory.shape[0]*int(len(trajs)), trajectory.shape[1]-3))
        all_trajectories = np.zeros((trajectory.shape[0]*s, trajectory.shape[1]-3))
        for i, time_bin in enumerate(trajs):
            if i < s:
                trajectory = loadmat(path_n + time_bin)
                trajectory = trajectory['trajectory'] 
                trajectory = np.delete(trajectory, nan_cols, 1)
                idx_2 = i*trajectory.shape[0]
                all_trajectories[idx_2:idx_2+trajectory.shape[0], 0:trajectory.shape[1]] = trajectory

        lengths.append(all_trajectories.shape[0])

        if n==0:
            rats = all_trajectories
        if n>0:
            rats = np.concatenate((rats, all_trajectories), axis=0)

print(rats.shape)

traj_5
traj_3
traj_4
traj_1
(432000, 57)


## K-S Test same dimensions

In [3]:
def probability(sequence, decimals=1):
    '''
    input: 
        - 1D sequence of rv observations-
    return: 
        - probability vector
    '''
    
    if sequence.shape[0] < sequence.shape[1]:
        sequence = np.transpose(sequence)
    
    # round input sequence to avoid sparse probability vector
    sequence = np.round(sequence, decimals)
    unique = np.unique(sequence, axis=0)
    n_triplets = len(unique)
    
    # fill probability vector
    prob_vector = np.zeros((n_triplets, 1))
    for row in sequence:
        occurrences = len(np.where(np.all(np.isclose(sequence, row), axis=1))[0])
        idx = np.where(np.all(np.isclose(unique, row), axis=1))
        if prob_vector[idx[0]] == 0:
            prob_vector[idx[0]] = occurrences/(sequence.shape[0])
    return prob_vector

In [9]:
trajectories = os.listdir(path)
n_rats = len(trajectories)

dims = np.arange(12,15)
print("CALCULATING PROBABILITY VECTORS")
for i in range(n_rats-1):
    #print(lengths[i])
    sequence = rats[0:lengths[i], dims]
    prob_vector = probability(sequence)
    if i == 0:
        probs = prob_vector
    else:
        probs = np.concatenate((probs, prob_vector), axis=1)

print("CALCULATING K-S MATRIX")
# "ks_matrix" is symmetric (A=A')
ks_matrix = np.zeros((len(trajectories)-1, len(trajectories)-1))
significance_level = 0.01
for ii in range(n_rats-1):
    for jj in range(n_rats-1):
        _, p_value = stats.ks_2samp(probs[:, ii], probs[:, jj])
        if p_value<significance_level:
            ks_matrix[ii, jj] = 0
        elif p_value>=significance_level:
            ks_matrix[ii, jj] = 1
            
print(trajectories)
print(ks_matrix)

CALCULATING PROBABILITY VECTORS
CALCULATING K-S MATRIX
['traj_5', 'traj_3', 'traj_4', 'traj_2', 'traj_1']
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]


In [ ]:
print(np.arange(1,50, 4))

[ 1  5  9 13 17 21 25 29 33 37 41 45 49]
